## 1. Data Prepare

In [8]:
from models.Trainer import *
dataset='mnist'
class_num=10

net_glob,train_loder,test_loader=get_net_and_loader(dataset=dataset,mode="ALL")

traindata=train_loder.dataset.train_data
targetdata=train_loder.dataset.train_labels

import os
from PIL import Image
from tqdm import tqdm

def save(x,i):
    #plt.figure(dpi=80)
    #data=np.transpose(x.cpu().numpy(), (1,2,0))
    data=x.cpu().numpy()
    # plt.imshow(data,cmap='gray') 
    # plt.axis('off')
    # plt.show()
    im = Image.fromarray(data)
    im.save(i)
    
dir_name='data/{}/train'.format(dataset)
    
for i in range(class_num):
    child_dir_name='{}/{}/'.format(dir_name,i)
    if not os.path.isdir(child_dir_name):
        mkdir_p(child_dir_name)
    
for i in tqdm(range(len(traindata))):
    #show(traindata[i],'{}/{}/{}_{}.jpg'.format(dir_name,targetdata[i],targetdata[i],i))
    pass


100%|██████████| 60000/60000 [00:00<00:00, 1828180.89it/s]


In [9]:
from models.sampling import *

class DatasetSplit(Dataset):
    def __init__(self, traindata,targetdata, idxs):
        self.traindata = traindata
        self.targetdata= targetdata
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        print(self.idxs[item])
        image, label = self.traindata[self.idxs[item]],self.targetdata[self.idxs[item]]
        return image, label
    
def save_dataset(dataset,dir_name):
    traindata = dataset.traindata
    targetdata= dataset.targetdata
    if not os.path.isdir(dir_name):
        mkdir_p(dir_name)
    
    for i in range(class_num):
        child_dir_name='{}/{}/'.format(dir_name,i)
        if not os.path.isdir(child_dir_name):
            mkdir_p(child_dir_name)
    
    for i in tqdm(range(len(traindata))):
        data=traindata[i].cpu().numpy()
        #im = Image.fromarray(data)
        #save(traindata[i],'{}/{}/{}_{}.jpg'.format(dir_name,targetdata[i],targetdata[i],i))

dict_users = mnist_iid(traindata, 4)

for idx in dict_users:
    x=DatasetSplit(traindata,targetdata, dict_users[idx])
    dir_name='test_data/{}/train/part_{}'.format(dataset,idx)
    #save_dataset(x,dir_name)

## 2. Socket test

In [11]:
import socket
sock=socket.socket()
sock.connect(("localhost",22))

In [13]:
import collections
import pandas as pd
from io import StringIO
import pickle
w=net_glob.cpu().state_dict()
w=dict(w)

#data = dict({key:value.numpy() for key,value in w.items()})
print(w.keys())
w1=pickle.dumps(w)
print(len(w1))
w2=pickle.loads(w1) 
print(w2.keys())
net_glob.load_state_dict(w2)


dict_keys(['layer_hidden.bias', 'layer_hidden.weight', 'layer_input.weight', 'layer_input.bias'])
204851
dict_keys(['layer_hidden.bias', 'layer_hidden.weight', 'layer_input.bias', 'layer_input.weight'])


## 3. Paillier

In [17]:
!pip install phe --user

Processing /home/dsjxtjc/2019211333/.cache/pip/wheels/f8/dc/36/dcb6bf0f1b9907e7b710ace63e64d08e7022340909315fdea4/phe-1.4.0-py2.py3-none-any.whl
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [82]:
from phe import paillier
from functools import reduce 
import numpy as np

def sum(listdata):
    return reduce(lambda x, y: x + y, listdata)

x=np.array([1.1,2.2,3.1])
print("origin: %s" % (sum(x)))
public_key,private_key=paillier.generate_paillier_keypair()
encrypt_data=[public_key.encrypt(i) for i in x]

print(type(x[0]))
encrypt_sum = sum(encrypt_data)
print("encrypt: %s" % (encrypt_sum))

decrypt_sum=private_key.decrypt(encrypt_sum)
print("decrypt: %s" % (decrypt_sum))


origin: 6.4
<class 'numpy.float64'>
encrypt: <phe.paillier.EncryptedNumber object at 0x7ff2a46d8e10>
decrypt: 6.4


In [108]:
net_glob,train_loder,test_loader=get_net_and_loader(dataset=dataset,mode="ALL")
w=net_glob.cpu().state_dict()

from tqdm import tqdm

for key,value in w.items():
    npv=value.numpy().flatten()
    for item in tqdm(npv):
        out=public_key.encrypt(item.astype(np.float64))
    #%time out = list(map(lambda item: public_key.encrypt(item.astype(np.float64)), npv))
    
    
    print(len(out))
    
    #size=npv.shape
# data = dict({key:public_key.encrypt(value.numpy()) for key,value in w.items()})
# data

 19%|█▊        | 9397/50176 [19:30<1:24:01,  8.09it/s]

KeyboardInterrupt: 

In [67]:
npv=value.numpy()
def f(x):
    print(x)
    return x
r = map(f, npv)
print(np.array(r))

In [104]:
npv=value.numpy().flatten()
out = list(map(lambda item: public_key.encrypt(item.astype(np.float64)), npv))
print(np.array(out))

10


In [105]:
%time out = list(map(lambda item: public_key.encrypt(item.astype(np.float64)), npv))

CPU times: user 1.3 s, sys: 4 ms, total: 1.31 s
Wall time: 1.3 s


In [102]:
%time out = list(public_key.encrypt(item.astype(np.float64)) for item in npv)

CPU times: user 1.29 s, sys: 0 ns, total: 1.29 s
Wall time: 1.28 s


In [ ]:
import numpy as np

In [26]:
import pickle
print(public_key)
send_data=pickle.dumps(public_key)
print(send_data)
public_key_worker=pickle.loads(send_data)
print(public_key_worker)

<PaillierPublicKey 746a13f920>
b'\x80\x03cphe.paillier\nPaillierPublicKey\nq\x00)\x81q\x01}q\x02(X\x07\x00\x00\x00nsquareq\x03\x8b\x00\x02\x00\x00\x19D3\xbf?\x81H\xfa4\x14\xcd\xaeI1\xd9\x07GJC\xea\xadC\xa5\x0c\x16\xa6\x93\x9aCv\xe3\x91\x97!\xd7\xcbC2\xe6\xaa\x9b\xaa\t\xba\xb1Q\x03\xc4\xc9\xf0L\xee\xb11\xe7\xee\xdc\xcc\xd7\x8d{\xe2\xad#F!\xb5o\xfd\x8dYp\x9e\x8dC\x91\x00\x1c\xc7\xb5\xa7f\xe9\t`\x0c{\\\x02 \xf1}\xb3\x99?\xf9\xd8\x9c\xe1R\xf3)\xb5\xfd%\r\xd2\x02\xb6L\xe0f\x0cwTL\x03\x15Fr~\x0b\xb8\x08?u\x82\xe0\'^\xa4\xc1\xc8\xe8\xbdz\x99\x14\xdb0Acn}\x9c\x82:\x86\x13J\x7fj@\x06\xf3\xb3k>\xda\xe8_}\x1b\xeb\xa1\x12\x12\xa1\xb6v2>\xe5\xf81\x91\x053\x05"88\x83\xb9\xeb\x1d\xf6\xfa\xf5\x9a\xc6\\\xfc\xc7\xe5\x98\x9e\xb7\xe4\xfa?%\xde\x04\xd2r(j\xc9\x0c C\xebc\xf74\xed\x8ai\xeee\xa0\x86U9\x18.\xd2x\x14\xc8\x11\x8a\x14^n#\x9f\x8e\xaaa\x07Z\xc5\xef\xb9\xd5%\xd2;\xa2d=\xa9{ \x8d\xd4\xabx\xabH\x01\xd2\xab\xb7\xe4\xbb\xfdQ7\xca\xe7\xcb\x0c\xce[\xa8#eA\x00\xb5o=\xd4B\xf5\x08,\x1b7\x80\xe9\xae\xfda\xc6q

## another method